In [1]:
import torch
import pickle
import pandas as pd
from torchtext import data
with open('preprocessed.pickle', 'rb') as f:
    X_train,X_test,y_train,y_test = pickle.load(f)

In [2]:
dic={}
dic['text']=[X_train]
dic['Label']=[y_train]
s=pd.Series([X_train,y_train])
df=pd.DataFrame.from_items(zip(s.index, s.values))

C:\Python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  """


In [3]:
df = df.rename(columns={0: 'text', 1: 'label'})

In [4]:
df['text'] = df['text'].apply(lambda x : ' '.join(word for word in x))

In [5]:
df.head()

,text,label
0,two qualifi right front actual think joe baker...,0
1,saw movi v afternoon see anyon sit piec trash ...,0
2,classic mani great dialog scene nobodi miss ni...,1
3,good origin 101 better cours cruella steal sho...,1
4,drawn movi curiou see adapt hubert shelbi brut...,0


In [6]:
TEXT = data.Field(tokenize=None,batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [7]:
fields = [(None, None), ('text',TEXT),('label', LABEL)]

In [8]:
df.to_csv(r'C:\Users\srika\OneDrive\Desktop\Summer20\Sentiment_Analysis_project\dataframes\df_new.csv')

In [9]:
training_data=data.TabularDataset(path = r'C:\Users\srika\OneDrive\Desktop\Summer20\Sentiment_Analysis_project\dataframes\df_new.csv',format = 'csv',fields = fields,skip_header = True)

In [10]:
print(vars(training_data.examples[0]))

{'text': ['two', 'qualifi', 'right', 'front', 'actual', 'think', 'joe', 'baker', 'good', 'even', 'great', 'right', 'materi', 'right', 'director', 'cape', 'fear', 'remak', 'small', 'role', 'goldeney', 'walk', 'tall', 'even', 'like', 'baker', 'mitchel', 'play', 'anti', 'hero', 'suppos', 'unlik', 'ye', 'mst3k', 'coverag', 'hilari', 'took', 'lot', 'cheap', 'shot', 'baker', 'deserv', 'keep', 'thing', 'live', 'entertain', 'appropri', 'level', 'tone', 'movi', 'best', 'part', 'movi', 'final', 'justic', 'seem', 'spite', 'exot', 'locat', 'cowboy', 'frontier', 'justic', 'theme', 'quit', 'bit', 'weaker', 'mitchel', 'main', 'reason', 'baker', 'charact', 'written', 'idiot', 'movi', 'conceit', 'baker', 'embodi', 'old', 'style', 'frontier', 'machismo', 'challeng', 'oppon', 'old', 'style', 'mano', 'mano', 'quick', 'draw', 'contest', 'tough', 'macho', 'alway', 'win', 'even', 'hurt', 'wound', 'outnumb', 'etc', 'conceit', 'lot', 'potenti', 'work', 'gari', 'cooper', 'even', 'condemn', 'film', 'b', 'movi', 

In [11]:
import random
SEED = 2020
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [15]:
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)  

Size of TEXT vocabulary: 22185
Size of LABEL vocabulary: 2
[('movi', 36057), ('film', 33685), ('one', 19423), ('like', 15914), ('time', 11212), ('good', 10797), ('make', 10571), ('charact', 9869), ('see', 9844), ('get', 9837)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000002295488E978>>, {'<unk>': 0, '<pad>': 1, 'movi': 2, 'film': 3, 'one': 4, 'like': 5, 'time': 6, 'good': 7, 'make': 8, 'charact': 9, 'see': 10, 'get': 11, 'watch': 12, 'stori': 13, 'even': 14, 'would': 15, 'realli': 16, 'well': 17, 'scene': 18, 'look': 19, 'show': 20, 'end': 21, 'much': 22, 'bad': 23, 'peopl': 24, 'go': 25, 'also': 26, 'great': 27, 'first': 28, 'love': 29, 'way': 30, 'think': 31, 'act': 32, 'play': 33, 'made': 34, 'thing': 35, 'could': 36, 'say': 37, 'know': 38, 'seem': 39, 'work': 40, 'plot': 41, 'two': 42, 'actor': 43, 'year': 44, 'mani': 45, 'come': 46, 'take': 47, 'seen': 48, 'life': 49, 'want': 50, 'best': 51, 'never': 52, 'tri': 53, 'littl': 54, 'ever

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [17]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [18]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 48
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [19]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(22185, 100)
  (lstm): LSTM(100, 48, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=96, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 2,332,261 trainable parameters
torch.Size([22185, 100])


In [20]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [21]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.493 | Train Acc: 75.06%
	 Val. Loss: 0.372 |  Val. Acc: 84.15%
	Train Loss: 0.279 | Train Acc: 88.74%
	 Val. Loss: 0.348 |  Val. Acc: 86.51%
	Train Loss: 0.175 | Train Acc: 93.66%
	 Val. Loss: 0.302 |  Val. Acc: 87.95%
	Train Loss: 0.112 | Train Acc: 96.23%
	 Val. Loss: 0.365 |  Val. Acc: 87.54%
	Train Loss: 0.064 | Train Acc: 98.01%
	 Val. Loss: 0.533 |  Val. Acc: 85.61%


In [29]:
path='saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();

#inference 
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    #print(tensor)
    tensor = tensor.unsqueeze(1)                #reshape in form of batch,no. of words
    tensor = tensor.transpose(0,1)
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)                  #prediction 
    return round(prediction.item()) 

In [30]:
predict(model, "one of the worst movies, I have ever seen")

0

In [33]:
predict(model, "The movie is so good, how come i never watched this movie")

1